In [1]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import random

from cnn import *
from Decoders import *
from utils import *

# device = torch.device("cpu")

MODEL TESTING

corpus

In [2]:
acchulu = ['అ', 'ఆ', 'ఇ', 'ఈ', 'ఉ', 'ఊ', 'ఋ', 'ౠ', 'ఌ', 'ౡ', 'ఎ', 'ఏ', 'ఐ', 'ఒ', 'ఓ', 'ఔ', 'అం', 'అః']
hallulu = ['క', 'ఖ', 'గ', 'ఘ', 'ఙ',
           'చ', 'ఛ', 'జ', 'ఝ', 'ఞ',
           'ట', 'ఠ', 'డ', 'ఢ', 'ణ',
           'త', 'థ', 'ద', 'ధ', 'న',
           'ప', 'ఫ', 'బ', 'భ', 'మ',
           'య', 'ర', 'ల', 'వ', 'శ', 'ష', 'స', 'హ', 'ళ', 'క్ష', 'ఱ']
vallulu = ['ా', 'ి', 'ీ', 'ు' , 'ూ', 'ృ', 'ౄ', 'ె', 'ే', 'ై', 'ొ', 'ో', 'ౌ', 'ం', 'ః', 'ఁ', 'ౕ', 'ౖ', 'ౢ' ]
connector = ['్']
numbers = ['౦', '౧', '౨', '౩', '౪', '౫', '౬', '౭', '౮', '౯']
splcharacters= [' ', '!', '"', '#', '$', '%', '&', "'", '(', ')',
              '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[',
              '\\', ']', '^', '_', '`', '{', '|', '}', '~', '1','2', '3', '4', '5', '6', '7', '8', '9', '0', 'ఽ']
spl = splcharacters + numbers

bases = acchulu + hallulu + spl
vms = vallulu
cms = hallulu

print("Bases: ", len(bases))
print("Vms: ", len(vms))
print("Cms: ", len(cms))

characters = bases+vms+cms+connector

base_mapping = {}
i = 2
for x in bases:
  base_mapping[x] = i
  i+=1

vm_mapping = {}
i = 2
for x in vms:
  vm_mapping[x] = i
  i+=1

cm_mapping = {}
i = 2
for x in cms:
  cm_mapping[x] = i
  i+=1

  
# creates a list of ductionaries with each dictionary reporesenting a term
def wordsDicts(s):
  List = []
  for i in range(len(s)):
    x = s[i]
    prev = ''
    if i > 0: prev = s[i-1]
    #----------------------------------is it a base term-----------------------
    if((x in acchulu or x in hallulu)  and prev != connector[0]):
      List.append({})
      List[-1]['base'] = x
    #----------------------------if it is a consonant modifier-----------------
    elif x in hallulu and prev == connector[0]:
      if(len(List) == 0):
        print(x)
      if('cm' not in List[-1]): List[-1]['cm'] = []
      List[len(List)-1]['cm'].append(x)

      #---------------------------if it is a vowel modifier--------------------
    elif x in vallulu:
      if(len(List) == 0):
        print(x)

      if('vm' not in List[-1]): List[-1]['vm'] = []
      List[len(List)-1]['vm'].append(x)

      #----------------------------it is a spl character-----------------------
    elif x in spl:
      List.append({})
      List[len(List)-1]['base'] = x
    else:
      continue
  return List

Bases:  108
Vms:  19
Cms:  36


index encoding

In [3]:
def index_encoding(s):
  List = wordsDicts(s)
  onehot = []
  for i in range(len(List)):
    D = List[i]
    onehotbase=  [1]
    onehotvm1 =  [1]
    onehotvm2 =  [1]
    onehotvm3 =  [1]
    onehotvm4 =  [1]
    onehotcm1 =  [1]
    onehotcm2 =  [1]
    onehotcm3 =  [1]
    onehotcm4 =  [1]


    onehotbase[0] = base_mapping[D['base']]

    it = 1
    if('vm' in D):
      for j in D['vm']:
        if it == 1:
          onehotvm1[0] = vm_mapping[j]
        elif it == 2:
          onehotvm2[0] = vm_mapping[j]
        elif it == 3:
          onehotvm3[0] = vm_mapping[j]
        elif it == 4:
          onehotvm4[0] = vm_mapping[j]
        it += 1
    
    it = 1
    if('cm' in D):
      for j in D['cm']:
        if it == 1:
          onehotcm1[0] = cm_mapping[j]
        elif it == 2:
          onehotcm2[0] = cm_mapping[j]
        elif it == 3:
          onehotcm3[0] = cm_mapping[j]
        elif it == 4:
          onehotcm4[0] = cm_mapping[j]
        it += 1
    onehoti = onehotbase + onehotvm1 + onehotvm2 + onehotvm3 + onehotvm4 + onehotcm1 + onehotcm2 + onehotcm3 + onehotcm4 #size 110 + 4*21 + 4*38 = 346
    onehot.append(onehoti)
  return torch.tensor(onehot)

def index_decoder(List):
  x = ""
  for onehoti in List:
    if onehoti[0] > 1:
      x += bases[onehoti[0]-2]

    if onehoti[5] > 1:
      x += connector[0]
      x += cms[onehoti[5]-2]
    if onehoti[6] > 1:
      x += connector[0]
      x += cms[onehoti[6]-2]
    if onehoti[7] > 1:
      x += connector[0]
      x += cms[onehoti[7]-2]
    if onehoti[8] > 1:
      x += connector[0]
      x += cms[onehoti[8]-2]

    if onehoti[1] > 1:
      x += vms[onehoti[1]-2]
    if onehoti[2] > 1:
      x += vms[onehoti[2]-2]
    if onehoti[3] > 1:
      x += vms[onehoti[3]-2]
    if onehoti[4] > 1:
      x += vms[onehoti[4]-2]
  return x

one-hot-encoding

In [4]:
def one_hot_encoder(s):
  List = wordsDicts(s)
  onehot = []
  for i in range(len(List)):
    D = List[i]
    onehotbase=  [0 for _ in range(len(bases) + 2)]
    onehotvm1 =  [0 for _ in range(len(vms) + 2)]
    onehotvm2 =  [0 for _ in range(len(vms) + 2)]

    onehotcm1 =  [0 for _ in range(len(cms) + 2)]   
    onehotcm2 =  [0 for _ in range(len(cms) + 2)]   

    onehotbase[base_mapping[D['base']]] = 1

    it = 1
    if('vm' in D):
      for j in D['vm']:
        if it == 1:
          onehotvm1[vm_mapping[j]] = 1
        elif it == 2:
          onehotvm2[vm_mapping[j]] = 1
        it += 1
    
    it = 1
    if('cm' in D):
      for j in D['cm']:
        if it == 1:
          onehotcm1[cm_mapping[j]] = 1
        elif it == 2:
          onehotcm2[cm_mapping[j]] = 1
        it += 1


    onehoti = onehotbase + onehotvm1 + onehotvm2 + onehotcm1 + onehotcm2 #size 110 + 4*21 + 4*38 = 346
    onehot.append(onehoti)
  encoded = torch.tensor(onehot).float().to(device)
  return encoded

def One_Hot_Decoder(List):
  x = ""
  for onehoti in List:
    for i in range(0, 110):
      if onehoti[i] == 1 and i > 1:
          x += bases[i-2]

    for i in range(152, 190):
      if onehoti[i] == 1 and i > 152:
        x += connector[0]
        x += cms[i-153]
    for i in range(190, 228):
      if onehoti[i] == 1 and i > 191:
        x += connector[0]
        x += cms[i-192]

    for i in range(110, 131):
      if onehoti[i] == 1 and i > 111:
        x += vms[i-112]
    for i in range(131, 152):
      if onehoti[i] == 1 and i > 132:
        x += vms[i-133]
  return x

In [5]:
def One_Hot_Decoder_Only_Base(List):
  x = ""
  for onehoti in List:
    if onehoti[0] == 1:
      x += '_'
      continue
    for i in range(0, 110):
      if onehoti[i] == 1 and i > 1:
          x += bases[i-2]
  return x

def One_Hot_Decoder_Only_Vm1(List):
  x = ""
  for onehoti in List:
    if onehoti[110] == 1:
      x += '_'
      continue
    if onehoti[111] == 1:
      x += ' '
      continue
    for i in range(110, 131):
      if onehoti[i] == 1 and i > 111:
        x += vms[i-112]
  return x

def One_Hot_Decoder_Only_Vm2(List):
  x = ""
  for onehoti in List:
    if onehoti[131] == 1:
      x += '_'
      continue
    if onehoti[132] == 1:
      x += ' '
      continue
    for i in range(131, 152):
      if onehoti[i] == 1 and i > 132:
        x += vms[i-133]
  return x


def One_Hot_Decoder_Only_Cm1(List):
  x = ""
  for onehoti in List:
    if onehoti[152] == 1:
      x += '_'
      continue
    if onehoti[153] == 1:
      x += ' '
      continue
    for i in range(152, 190):
      if onehoti[i] == 1 and i > 153:
        x += connector[0]
        x += cms[i-153]
  return x

def One_Hot_Decoder_Only_Cm2(List):
  x = ""
  for onehoti in List:
    if onehoti[190] == 1:
      x += '_'
      continue
    if onehoti[190] == 1:
      x += ' '
      continue
    for i in range(190, 228):
      if onehoti[i] == 1 and i > 191:
        x += connector[0]
        x += cms[i-192]
  return x

def index_decoder_Only_Base(List):
  x = ""
  for onehoti in List:
    if onehoti[0] > 1:
      x += bases[onehoti[0]-2]
  return x

In [ ]:
cnn = EncoderCNN().to(device)
rnn = DECODER_RNN().to(device)

cnn.load_state_dict(torch.load("/home/ocr/teluguOCR/Models/Best_CNN/GRU1/Model5.pth"))
rnn.load_state_dict(torch.load("/home/ocr/teluguOCR/Models/Best_RNN/GRU1/Model5.pth"))

cnn.eval()
rnn.eval()

In [ ]:
# load a JPEG image an convert to tensor
from PIL import Image
import torch
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import cv2 as cv

img = Image.open("/home/ocr/teluguOCR/test7.png")
img = img.convert('L')

# Denoising the image using Gaussian filter
img = cv.fastNlMeansDenoising(np.array(img), None, 10, 7, 21)

img = transforms.ToTensor()(img)
# img = image_tensor[0, 610:690, 160:405]

print(img.shape)
print(torch.max(img), torch.min(img))
img *= 255

img = Image.fromarray(np.array(img[0], dtype=np.uint8))

m = img.size[1]//40

img = img.resize((img.size[0]//m, 40))

img = transforms.ToTensor()(img)
img [img< 0] = 0
img [img> 255] = 255

img*= 255

img = 255 - img

th = 100
img[img > th] = 255
# img[img < th] = 0

f_img = torch.zeros(40, 800)

f_img[:, 0:img.shape[2]] = img[0]

plt.figure(figsize=(10,10))
plt.imshow(f_img[:, :], cmap='gray')
plt.show()

In [ ]:
cnn_out = cnn(f_img.unsqueeze(0).unsqueeze(0).to(device)).unsqueeze(1)

f_out = rnn(cnn_out)

f_out[:, :, 0:110] = F.softmax(f_out[:, :, 0:110], dim=2)

f_out[:, :, 110:131] = F.softmax(f_out[:, :, 110:131], dim=2)
f_out[:, :, 131:152] = F.softmax(f_out[:, :, 131:152], dim=2)

f_out[:, :, 152:190] = F.softmax(f_out[:, :, 152:190], dim=2)
f_out[:, :, 190:229] = F.softmax(f_out[:, :, 190:228], dim=2)

f_out = f_out.squeeze(1)

final_label = torch.zeros(Image_length, Text_embedding_size).to(device)

for i in range(Image_length):
    final_label[i, torch.argmax(f_out[i, :110])] = 1
    final_label[i, 110 + torch.argmax(f_out[i, 110:131])] = 1
    final_label[i, 131 + torch.argmax(f_out[i, 131:152])] = 1
    final_label[i, 152 + torch.argmax(f_out[i, 152:190])] = 1
    final_label[i, 190 + torch.argmax(f_out[i, 190:228])] = 1


print("predicted label: ", One_Hot_Decoder(final_label))


In [ ]:
for i in range(100):
    if(final_label[i][0] == 1):
        print('_', end = '')
    else:
        print(One_Hot_Decoder_Only_Base(final_label[i][:110].unsqueeze(0)), end = '')
print()

base = []
vm1 = []
vm2 = []
cm1 = []
cm2 = []

for i in range(100):
    base.append(One_Hot_Decoder_Only_Base(final_label[i].unsqueeze(0)))
    vm1.append(One_Hot_Decoder_Only_Vm1(final_label[i].unsqueeze(0)))
    vm2.append(One_Hot_Decoder_Only_Vm2(final_label[i].unsqueeze(0)))
    cm1.append(One_Hot_Decoder_Only_Cm1(final_label[i].unsqueeze(0)))
    cm2.append(One_Hot_Decoder_Only_Cm2(final_label[i].unsqueeze(0)))

# print(base)
# print(vm1)
# print(vm2)
# print(vm3)
# print(vm4)
# print(cm1)
# print(cm2)
# print(cm3)
# print(cm4)

# Removing the continuous duplicate occurances of the same character in the Lists
base = [base[i] for i in range(len(base)) if i == 0 or base[i] != base[i-1]]
vm1 = [vm1[i] for i in range(len(vm1)) if i == 0 or vm1[i] != vm1[i-1]]
vm2 = [vm2[i] for i in range(len(vm2)) if i == 0 or vm2[i] != vm2[i-1]]
cm1 = [cm1[i] for i in range(len(cm1)) if i == 0 or cm1[i] != cm1[i-1]]
cm2 = [cm2[i] for i in range(len(cm2)) if i == 0 or cm2[i] != cm2[i-1]]

# removing all occurances fo '' in the Lists
base = [x for x in base if x != '_']
vm1 = [x for x in vm1 if x != '_']
vm2 = [x for x in vm2 if x != '_']
cm1 = [x for x in cm1 if x != '_']
cm2 = [x for x in cm2 if x != '_']

print(len(base), " | ", base)
print(len(vm1), " | ", vm1)
print(len(vm2), " | ", vm2)
print(len(cm1), " | ", cm1)
print(len(cm2), " | ", cm2)

x = ""
for i in range(len(base)):
    x += base[i]
    if base[i] == ' ':
      continue

    if i < len(cm1) and  cm1[i] != ' ':
      x += cm1[i]
    if i < len(cm2) and  cm2[i] != ' ':
      x += cm2[i]

    if i < len(vm1) and  vm1[i] != ' ':
      x += vm1[i]
    if i < len(vm2) and  vm2[i] != ' ':
      x += vm2[i]

plt.figure(figsize=(10,10))
plt.imshow(f_img[:, :], cmap='gray')
plt.show()

print(x)

Image from data set

In [ ]:
image_num = random.randint(1, 150854 + 1)

from dataset import *

data = TeluguOCRDataset("/home/ocr/teluguOCR/Dataset/Cropped_Dataset/Images", "/home/ocr/teluguOCR/Dataset/Cropped_Dataset/Labels")

data_point = data.__getitem__(image_num)

image = data_point[0]
label = data_point[1]

image = image.to(device)
label = label.to(device)

plt.figure(figsize=(10,10))
plt.imshow(image.squeeze(0).squeeze(0).cpu(), cmap='gray')
plt.show()

label = label.squeeze(0).cpu().to(torch.int64).numpy()

print("actual label: ", index_decoder(label))

image = image.unsqueeze(0).to(device)

cnn_output = cnn(image).unsqueeze(1)


f_output = rnn(cnn_output)

f_output[:, :, 0:110] = F.softmax(f_output[:, :, 0:110], dim=2)

f_output[:, :, 110:131] = F.softmax(f_output[:, :, 110:131], dim=2)
f_output[:, :, 131:152] = F.softmax(f_output[:, :, 131:152], dim=2)

f_output[:, :, 152:190] = F.softmax(f_output[:, :, 152:190], dim=2)
f_output[:, :, 190:228] = F.softmax(f_output[:, :, 190:228], dim=2)

f_output = f_output.squeeze(1)

final_label = torch.zeros(Image_length, Text_embedding_size).to(device)

for i in range(Image_length):
    final_label[i, torch.argmax(f_output[i, :110])] = 1

    final_label[i, torch.argmax(f_output[i, 110:131])+110] = 1
    final_label[i, torch.argmax(f_output[i, 131:152])+131] = 1

    final_label[i, torch.argmax(f_output[i, 152:190])+152] = 1
    final_label[i, torch.argmax(f_output[i, 190:228])+190] = 1

print("predicted label: ", One_Hot_Decoder(final_label))

In [ ]:
# printing the range values in f_output
print(torch.max(f_output[0, :110]))
print(torch.min(f_output[0, :110]))

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(image.squeeze(0).squeeze(0).cpu(), cmap='gray')
plt.show()

print("   Actual label: ", index_decoder(label))

# for i in range(100):
#     if(final_label[i][0] == 1):
#         print('_', end = '')
#     else:
#         print(One_Hot_Decoder_Only_Base(final_label[i][:110].unsqueeze(0)), end = '')
# print()

base = []
vm1 = []
vm2 = []
vm3 = []
vm4 = []
cm1 = []
cm2 = []
cm3 = []
cm4 = []

for i in range(100):
    base.append(One_Hot_Decoder_Only_Base(final_label[i].unsqueeze(0)))
    vm1.append(One_Hot_Decoder_Only_Vm1(final_label[i].unsqueeze(0)))
    vm2.append(One_Hot_Decoder_Only_Vm2(final_label[i].unsqueeze(0)))
    cm1.append(One_Hot_Decoder_Only_Cm1(final_label[i].unsqueeze(0)))
    cm2.append(One_Hot_Decoder_Only_Cm2(final_label[i].unsqueeze(0)))

# print(base)
# print(vm1)
# print(vm2)
# print(vm3)
# print(vm4)
# print(cm1)
# print(cm2)
# print(cm3)
# print(cm4)

# Removing the continuous duplicate occurances of the same character in the Lists
base = [base[i] for i in range(len(base)) if i == 0 or base[i] != base[i-1]]
vm1 = [vm1[i] for i in range(len(vm1)) if i == 0 or vm1[i] != vm1[i-1]]
vm2 = [vm2[i] for i in range(len(vm2)) if i == 0 or vm2[i] != vm2[i-1]]
cm1 = [cm1[i] for i in range(len(cm1)) if i == 0 or cm1[i] != cm1[i-1]]
cm2 = [cm2[i] for i in range(len(cm2)) if i == 0 or cm2[i] != cm2[i-1]]

# removing all occurances fo '' in the Lists
base = [x for x in base if x != '_']
vm1 = [x for x in vm1 if x != '_']
vm2 = [x for x in vm2 if x != '_']
cm1 = [x for x in cm1 if x != '_']
cm2 = [x for x in cm2 if x != '_']

x = ""
for i in range(len(base)):
    x += base[i]
    if base[i] == ' ':
      continue

    if i < len(cm1) and  cm1[i] != ' ':
      x += cm1[i]
    if i < len(cm2) and  cm2[i] != ' ':
      x += cm2[i]

    if i < len(vm1) and  vm1[i] != ' ':
      x += vm1[i]
    if i < len(vm2) and  vm2[i] != ' ':
      x += vm2[i]
      
print("Predicted Label: ", x, end = '\n\n')

print(len(base), " | ", base)
print(len(vm1), " | ", vm1)
print(len(vm2), " | ", vm2)
print(len(cm1), " | ", cm1)
print(len(cm2), " | ", cm2)

In [10]:
def get_accuracy(GTT, FP):
    GTT = GTT
    FP = FP
    
    T = len(wordsDicts(index_decoder(GTT)))
    B = 0
    V = 0
    C = 0
    for i in range(min(T, len(FP))):
        if GTT[i][0] == FP[i][0]:
                B += 1
        if GTT[i][1] == FP[i][1]:
                V += 1
        if GTT[i][2] == FP[i][2]:
                V += 1
        if GTT[i][5] == FP[i][5]:
                C += 1
        if GTT[i][6] == FP[i][6]:
                C += 1
    return T, B, C, V

In [11]:
def calculate_acc(cnn, rnn, image, label):
    label = label.squeeze(0).cpu().to(torch.int64).numpy()
    image = image.to(device)
    cnn_output = cnn(image).unsqueeze(1)
    f_output = rnn(cnn_output)

    f_output[:, :, 0:110] = F.softmax(f_output[:, :, 0:110], dim=2)
    f_output[:, :, 110:131] = F.softmax(f_output[:, :, 110:131], dim=2)
    f_output[:, :, 131:152] = F.softmax(f_output[:, :, 131:152], dim=2)
    f_output[:, :, 152:190] = F.softmax(f_output[:, :, 152:190], dim=2)
    f_output[:, :, 190:228] = F.softmax(f_output[:, :, 190:228], dim=2)

    f_output = f_output.squeeze(1)
    final_label = torch.zeros(Image_length, Text_embedding_size).to(device)

    for i in range(Image_length):
        final_label[i, torch.argmax(f_output[i, :110])] = 1
        final_label[i, torch.argmax(f_output[i, 110:131])+110] = 1
        final_label[i, torch.argmax(f_output[i, 131:152])+131] = 1
        final_label[i, torch.argmax(f_output[i, 152:190])+152] = 1
        final_label[i, torch.argmax(f_output[i, 190:228])+190] = 1

    base = []
    vm1 = []
    vm2 = []
    cm1 = []
    cm2 = []

    for i in range(100):
        base.append(One_Hot_Decoder_Only_Base(final_label[i].unsqueeze(0)))
        vm1.append(One_Hot_Decoder_Only_Vm1(final_label[i].unsqueeze(0)))
        vm2.append(One_Hot_Decoder_Only_Vm2(final_label[i].unsqueeze(0)))
        cm1.append(One_Hot_Decoder_Only_Cm1(final_label[i].unsqueeze(0)))
        cm2.append(One_Hot_Decoder_Only_Cm2(final_label[i].unsqueeze(0)))

    # Removing the continuous duplicate occurances of the same character in the Lists
    base = [base[i] for i in range(len(base)) if i == 0 or base[i] != base[i-1]]
    vm1 = [vm1[i] for i in range(len(vm1)) if i == 0 or vm1[i] != vm1[i-1]]
    vm2 = [vm2[i] for i in range(len(vm2)) if i == 0 or vm2[i] != vm2[i-1]]
    cm1 = [cm1[i] for i in range(len(cm1)) if i == 0 or cm1[i] != cm1[i-1]]
    cm2 = [cm2[i] for i in range(len(cm2)) if i == 0 or cm2[i] != cm2[i-1]]

    # removing all occurances fo '' in the Lists
    base = [x for x in base if x != '_']
    vm1 = [x for x in vm1 if x != '_']
    vm2 = [x for x in vm2 if x != '_']
    cm1 = [x for x in cm1 if x != '_']
    cm2 = [x for x in cm2 if x != '_']
    
    x = ""
    for i in range(len(base)):
        x += base[i]
        if base[i] == ' ':
            continue

        if i < len(cm1) and  cm1[i] != ' ':
            x += cm1[i]
        if i < len(cm2) and  cm2[i] != ' ':
            x += cm2[i]

        if i < len(vm1) and  vm1[i] != ' ':
            x += vm1[i]
        if i < len(vm2) and  vm2[i] != ' ':
            x += vm2[i]
    
    T, B, C, V = get_accuracy(label, index_encoding(x))
    acc = (0.5*B + 0.3*(C//2) + 0.2*(V//2))/T
    return acc

In [12]:
from dataset import *
dataset = TeluguOCRDataset("/home/ocr/teluguOCR/Dataset/Cropped_Dataset/Images", "/home/ocr/teluguOCR/Dataset/Cropped_Dataset/Labels")

# splitting the dataset into training and validation
torch.manual_seed(0)
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [0.7, 0.3])

train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=1, shuffle=True)

cnn = EncoderCNN().to(device)
rnn = DECODER_RNN().to(device)

cnn.load_state_dict(torch.load("/home/ocr/teluguOCR/Models/Best_CNN/LSTM/Model5.pth"))
rnn.load_state_dict(torch.load("/home/ocr/teluguOCR/Models/Best_RNN/LSTM/Model5.pth"))

# train accuracy
acc = 0
total_num = 0
for image, label, _, _ in train_dataloader:
    acc += calculate_acc(cnn, rnn, image, label)
    total_num += 1
    if total_num == 1000:
        break
    print("T: ", total_num, end = '\r')
print("Train Accuracy: ", (acc/total_num))

# validation accuracy
acc = 0
total_num = 0
for image, label, _, _ in val_dataloader:
    acc += calculate_acc(cnn, rnn, image, label)
    total_num += 1
    if total_num == 1000:
        break
    print("V: ", total_num, end = '\r')
print("Validation Accuracy: ", (acc/total_num))

Train Accuracy:  0.7687097757353727
Validation Accuracy:  0.7638860313946141
